# Kernels for the one loop power spectrum, bispectrum, and response kernels in SPT

In [1]:
import numpy as np
import sympy as sp

import sys
import vectors as vect
import kernels

Some code to convert the resulting kernels into C format.

## Tree level bispectrum kernel

As a warmup, the tree level bispectrum kernel.

In [2]:
vk1, vk2 = vect.vectors("k1", "k2")
k1, k2, k3 = sp.symbols(["k1", "k2", "k3"])
f2_raw = kernels.Fs(2, [vk1, vk2])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
replacements = [
    (k1dk1, k1**2), (k2dk2, k2**2), (k1dk2, (k3**2 - k1**2 - k2**2)/2)
]
f2_symbolic = sp.simplify(vect.sympy(f2_raw).subs(replacements))
f2_string = str(f2_symbolic)
with open("tree_kernel.txt", "w") as fout:
    fout.write(f2_string)

We now want to take limits, it's easier to do it keeping the angle

In [3]:
vk1, vk2 = vect.vectors("k1", "k2")
k1, k2, mu12 = sp.symbols(["k1", "k2", "\mu_{12}"])
kernel_raw = kernels.Fs(2, [vk1, vk2])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
replacements = [
    (k1dk1, k1**2), (k2dk2, k2**2), (k1dk2, k1*k2*mu12)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series = sp.simplify(
    sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO().subs([(mu12, (k3**2 - k2**2 - k1**2)/(2*k1*k2))])
)
series_string = str(series)
with open("tree_kernel_squeezed.txt", "w") as fout:
    fout.write(series_string)


Now the squeezed kernel minus the consistency relation. (The response coefficients come from this.)

In [4]:
vk1, vk2 = vect.vectors("k1", "k2")
k1, k2, mu12 = sp.symbols(["k1", "k2", "\mu_{12}"])
kernel_raw = kernels.Fs(2, [vk1, vk2])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
replacements = [
    (k1dk1, k1**2), (k2dk2, k2**2), (k1dk2, k1*k2*mu12)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series_0 = sp.simplify(
    (sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO() -
    sp.series(
        kernel_symbolic, k1, 0, n=0
    ).removeO()
    ).subs([(mu12, (k3**2 - k2**2 - k1**2)/(2*k1*k2))])
)
series_string = str(series_0)
with open("tree_kernel_squeezed_0.txt", "w") as fout:
    fout.write(series_string)

Let's extract the response coefficients.

In [5]:
vk1, vk2 = vect.vectors("k1", "k2")
k1, k2, mu12 = sp.symbols(["k1", "k2", "\mu_{12}"])
kernel_raw = kernels.Fs(2, [vk1, vk2])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
replacements = [
    (k1dk1, k1**2), (k2dk2, k2**2), (k1dk2, k1*k2*mu12)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series_0 = sp.simplify(
    (sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO() -
    sp.series(
        kernel_symbolic, k1, 0, n=0
    ).removeO()
    )
)

result_poly = sp.poly(series_0, mu12)

b1_kernel = result_poly.coeff_monomial(1)
bmu_kernel = result_poly.coeff_monomial(mu12**2)

if result_poly.coeff_monomial(mu12) != 0 or result_poly.degree() > 2:
    raise Exception("Unexpected coefficient in mu12 polynomial!")

b1_kernel_string = str(b1_kernel)
bmu_kernel_string = str(bmu_kernel)

with open("tree_b1_kernel.txt", "w") as fin:
    fin.write(b1_kernel_string)
with open("tree_bmu_kernel.txt", "w") as fin:
    fin.write(bmu_kernel_string)

## Power spectrum loop kernels

Now the power spectrum kernels

### $P_{13}$ kernel

In [6]:
vk, vp, vv = vect.vectors("k", "p", "v")
eps = vect.Scalar("epsilon")
k, p, mu, epsilon = sp.symbols(["k", "p", "mu", "epsilon"])
f3_raw = kernels.Fs(3, [vk, vp, -vp + eps*vv])
kdp = vect.sympy(vect.Dot(vk, vp))
kdk = vect.sympy(vect.Dot(vk, vk))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (kdp, k*p*mu),
    (kdk, k**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(
    sp.series(
        vect.sympy(f3_raw), epsilon, 0, n=1
    ).removeO().subs(replacements)
)
kernel_string = str(kernel_symbolic)
with open("p31_kernel.txt", "w") as fout:
    fout.write(kernel_string)

### $P_{22}$ kernel

In [7]:
vk, vp = vect.vectors("k", "p")
k, p, mu = sp.symbols(["k", "p", "mu"])
f2_raw = kernels.Fs(2, [vk - vp, vp])
kdp = vect.sympy(vect.Dot(vk, vp))
kdk = vect.sympy(vect.Dot(vk, vk))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (kdp, k*p*mu),
    (kdk, k**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify((vect.sympy(f2_raw).subs(replacements))**2)
kernel_string = str(kernel_symbolic)
with open("p22_kernel.txt", "w") as fout:
    fout.write(kernel_string)

## Bispectrum loop kernels

### $B_{222}$ kernel

$\vec{k}_1 = (0, 0, k_1)$, 
$\vec{k}_2 = (k_2 \sin \theta_{12}, 0, k_2 \cos \theta_{12})$, 
$\vec{p} = (p \sin\theta \cos\phi, p \sin\theta \sin\phi, p\cos\theta)$.

$\vec{k}_1.\vec{p} = k_1 p \cos\theta$,

$\vec{k}_2.\vec{p} = k_2 p (\sin\theta \cos\phi \sin\theta_{12} + \cos\theta \cos\theta_{12})$,

$\cos\theta_{12} = \frac{1}{2 k_1 k_2} (k_3^2 - k_1^2 - k_2^2)$,

$\sin\theta_{12} = \frac{1}{2 k_1 k_2}\sqrt{4 k_1^2 k_2^2 - (k_3^2 - k_1^2 - k_2^2)^2}$

In [8]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
k1, k2, k3, p, mu1, mu2 = sp.symbols("k1 k2 k3 p mu1 mu2")
kernel_raw = kernels.Fs(2, [vk1 - vp, vp])*kernels.Fs(2, [vk2 + vp, -vp])*kernels.Fs(2, [vk2 + vp, vk1 - vp])
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2),
    (k1dk2, (k3**2 - k1**2 - k2**2)/2),
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))
kernel_string = str(kernel_symbolic)
with open("b222_kernel.txt", "w") as fout:
    fout.write(kernel_string)

Since $B_{222}$ goes to zero as $k_1^2$ in the squeezed limit $k_1 \rightarrow 0$, it doesn't contribute there.

### $B_{321}^I$ kernel

In [9]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
k1, k2, k3, p, mu1, mu2 = sp.symbols("k1 k2 k3 p mu1 mu2")
kernel_raw = kernels.Fs(3, [vk1, vk2 - vp, vp])*kernels.Fs(2, [vk2 - vp, vp])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2),
    (k1dk2, (k3**2 - k1**2 - k2**2)/2),
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))
kernel_string = str(kernel_symbolic)
with open("b321I_kernel.txt", "w") as fout:
    fout.write(kernel_string)

We now want to take limits, it's easier to do it keeping the angle

In [10]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
k1, k2, k3, mu12, p, mu1, mu2 = sp.symbols("k1 k2 k3 mu12 p mu1 mu2")
kernel_raw = kernels.Fs(3, [vk1, vk2 - vp, vp])*kernels.Fs(2, [vk2 - vp, vp])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2),
    (k1dk2, k1*k2*mu12),
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series = (
    sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO().subs([(mu12, (k3**2 - k1**2 - k2**2)/(2*k1*k2))])
)
series_string = str(series)
with open("b321I_kernel_squeezed.txt", "w") as fout:
    fout.write(series_string)


In [11]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
k1, k2, k3, mu12, p, mu1, mu2 = sp.symbols("k1 k2 k3 mu12 p mu1 mu2")
kernel_raw = kernels.Fs(3, [vk1, vk2 - vp, vp])*kernels.Fs(2, [vk2 - vp, vp])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2),
    (k1dk2, k1*k2*mu12),
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series = (
    (sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO() -
    sp.series(
        kernel_symbolic, k1, 0, n=0
    ).removeO()).subs([(mu12, (k3**2 - k2**2 - k1**2)/(2*k1*k2))])
)
series_string = str(series)
with open("b321I_kernel_squeezed_0.txt", "w") as fout:
    fout.write(series_string)

$$
\vec{k}_1 = k_1 (\sqrt{1 - \mu^2}, 0, \mu)\,,\quad
\vec{k}_2 = k_2 (0, 0, 1)\,,\quad
\vec{p} = p (\sqrt{1 - \mu_p^2}\cos\phi, \sqrt{1 - \mu_p^2}\sin\phi, \mu_p)
$$

$$
\mu_2 = \mu_p\,,\quad
\mu_1 = \sqrt{1 - \mu^2}\sqrt{1 - \mu_p^2}\cos\phi + \mu\mu_p
$$

In [5]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
k1, k2, k3, mu12, p, mu1, mu2 = sp.symbols("k1 k2 k3 mu12 p mu1 mu2")
kernel_raw = kernels.Fs(3, [vk1, vk2 - vp, vp])*kernels.Fs(2, [vk2 - vp, vp])
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
replacements = [
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2),
    (k1dk2, k1*k2*mu12),
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))

series = (
    (sp.series(
        kernel_symbolic, k1, 0, n=1
    ).removeO() -
    sp.series(
        kernel_symbolic, k1, 0, n=0
    ).removeO())
)

series_poly = sp.poly(series, mu1, mu12)

phi = sp.Symbol("phi")
phi_integral_mu1 = sp.integrate(
    sp.sqrt(1 - mu12**2)*sp.sqrt(1 - mu2**2)*sp.cos(phi) + mu2*mu12, 
(phi, 0, 2*sp.pi))
phi_integral_mu1_sq = sp.integrate(
    (sp.sqrt(1 - mu12**2)*sp.sqrt(1 - mu2**2)*sp.cos(phi) + mu2*mu12)**2, 
(phi, 0, 2*sp.pi))
phi_integral_const = 2*sp.pi

quadratic = series_poly.coeff_monomial(mu1**2)
linear = series_poly.coeff_monomial(mu1*mu12)*mu12
zeroth = series_poly.coeff_monomial(mu12**2)*mu12**2 + series_poly.coeff_monomial(1)

if (
    series_poly.coeff_monomial(mu1) != 0 
    or series_poly.coeff_monomial(mu12) != 0
    or series_poly.degree() > 2
):
    raise Exception("Polynomial has unexpected coefficients!")

result_poly = sp.poly(
    zeroth*phi_integral_const + linear*phi_integral_mu1 + quadratic*phi_integral_mu1_sq,
    mu12)

b1_kernel = sp.apart(result_poly.coeff_monomial(1), k2)
bmu_kernel = sp.apart(result_poly.coeff_monomial(mu12**2), k2)

if result_poly.coeff_monomial(mu12) != 0 or result_poly.degree() > 2:
    raise Exception("Unexpected coefficient in mu12 polynomial!")

b1_kernel_string = str(b1_kernel)
bmu_kernel_string = str(bmu_kernel)

with open("b321I_b1_kernel.txt", "w") as fin:
    fin.write(b1_kernel_string)
with open("b321I_bmu_kernel.txt", "w") as fin:
    fin.write(bmu_kernel_string)



### $B_{411}$ kernel

For $B_{411}$, the naive approach is too slow, producing a kernel which is simply too large. We need a better way to compute it. Let's take a look at the recursion relation

$$
\begin{align*}
&(33\times 24)F_4(\vec{k}_1, \vec{k}_2, \vec{k}_3, \vec{k}_4) = \\
&9\alpha(\vec{k}_1, \vec{k}_2 + \vec{k}_3 + \vec{k}_4) F_3(\vec{k}_2, \vec{k}_3, \vec{k}_4) + 2\beta(\vec{k}_1, \vec{k}_2 + \vec{k}_3 + \vec{k}_4) G_3(\vec{k}_2, \vec{k}_3, \vec{k}_4)\\
& + 9\alpha(\vec{k}_1 + \vec{k}_2, \vec{k}_3 + \vec{k}_4) G_2(\vec{k}_1, \vec{k}_2)F_2(\vec{k}_3, \vec{k}_4) + 2\beta(\vec{k}_1 + \vec{k}_2, \vec{k}_3 + \vec{k}_4) G_2(\vec{k}_1, \vec{k}_2)G_2(\vec{k}_2, \vec{k}_3)\\
&+ 9\alpha(\vec{k}_1 + \vec{k}_2 + \vec{k}_3, \vec{k}_4) G_3(\vec{k}_1, \vec{k}_2, \vec{k}_3) + 2\beta(\vec{k}_1 + \vec{k}_2 + \vec{k}_3, \vec{k}_4) G_3(\vec{k}_1, \vec{k}_2, \vec{k}_3)\\
& + 23\,\text{perms.}
\end{align*}
$$

Remember that $\alpha(\vec{k}_1, \vec{k}_2) = \vec{k}_{12}.\vec{k}_1/k_1^2$, and $\beta(\vec{k}_1, \vec{k}_2) = k_{12}^2 (\vec{k}_1.\vec{k}_2)/k_1^2 k_2^2$ which seems to diverge when one of the momenta goes to zero. But this appears multiplying a kernel that also vanishes quadratically:

In [14]:
k, v = vect.vectors("k", "v")
epsilon = vect.Scalar("epsilon")
eps = sp.Symbol("epsilon")

g2 = vect.sympy(kernels.Gs(2, [k, epsilon*v - k]))
sp.series(g2, eps, 0, n=3)

epsilon**2*(-\vec{v} \cdot \vec{v}/(14*\vec{k} \cdot \vec{k}) - 3*\vec{k} \cdot \vec{v}**2/(7*\vec{k} \cdot \vec{k}**2)) + O(epsilon**3)

In [15]:
f2 = vect.sympy(kernels.Fs(2, [k, epsilon*v - k]))
sp.series(f2, eps, 0, n=3)

epsilon**2*(3*\vec{v} \cdot \vec{v}/(14*\vec{k} \cdot \vec{k}) - 5*\vec{k} \cdot \vec{v}**2/(7*\vec{k} \cdot \vec{k}**2)) + O(epsilon**3)

We also need the regularized $F_3$ and $G_3$ in order to compute $F_4$.

$$
\begin{align*}
(18\times 6)F_3(\vec{k}_1, \vec{k}_2, \vec{k}_3) &= 7\alpha(\vec{k}_1, \vec{k}_2 + \vec{k}_3) F_2(\vec{k}_2, \vec{k}_3) + 2\beta(\vec{k}_1, \vec{k}_2 + \vec{k}_3) G_2(\vec{k}_2, \vec{k}_3)\\
& + G_2(\vec{k}_1, \vec{k}_2)\left(7\alpha(\vec{k}_1 + \vec{k}_2, \vec{k}_3) + 2\beta(\vec{k}_1 + \vec{k}_2, \vec{k}_3)\right)\\
& + 5\,\text{perms.}
\end{align*}
$$

$$
\begin{align*}
(18\times 6)G_3(\vec{k}_1, \vec{k}_2, \vec{k}_3) &= 3\alpha(\vec{k}_1, \vec{k}_2 + \vec{k}_3) G_2(\vec{k}_2, \vec{k}_3) + 6\beta(\vec{k}_1, \vec{k}_2 + \vec{k}_3) G_2(\vec{k}_2, \vec{k}_3)\\
& + G_2(\vec{k}_1, \vec{k}_2)\left(3\alpha(\vec{k}_1 + \vec{k}_2, \vec{k}_3) + 6\beta(\vec{k}_1 + \vec{k}_2, \vec{k}_3)\right)\\
& + 5\,\text{perms.}
\end{align*}
$$

In [16]:
from itertools import permutations

v0 = vect.Vector('0')

def F3_reg(ks):
    if ks[0] + ks[1] == v0:
        return (
            7*kernels.alpha(ks[0], ks[1] + ks[2])*kernels.Fs(2, ks[1:])
            + 2*kernels.beta(ks[0], ks[1] + ks[2])*kernels.Gs(2, ks[1:])
        )/18
    elif ks[1] + ks[2] == v0:
        return (
            kernels.Gs(2, ks[:2])*(7*kernels.alpha(ks[0] + ks[1], ks[2]) 
            + 2*kernels.beta(ks[0] + ks[1], ks[2]))
        )/18
    else:
        return kernels.F(3, ks)

def G3_reg(ks):
    if ks[0] + ks[1] == v0:
        return (
            3*kernels.alpha(ks[0], ks[1] + ks[2])*kernels.Gs(2, ks[1:])
            + 6*kernels.beta(ks[0], ks[1] + ks[2])*kernels.Gs(2, ks[1:])
        )/18
    elif ks[1] + ks[2] == v0:
        return (
            kernels.Gs(2, ks[:2])*(3*kernels.alpha(ks[0] + ks[1], ks[2]) 
            + 6*kernels.beta(ks[0] + ks[1], ks[2]))
        )/18
    else:
        return kernels.G(3, ks)

def F3s_reg(ks):
    return sum(map(lambda x: F3_reg(x), permutations(ks)))/6

def G3s_reg(ks):
    return sum(map(lambda x: G3_reg(x), permutations(ks)))/6


$$
\begin{align*}
&(33\times 24)F_4(\vec{k}_1, \vec{k}_2, \vec{k}_3, \vec{k}_4) = \\
&9\alpha(\vec{k}_1, \vec{k}_2 + \vec{k}_3 + \vec{k}_4) F_3(\vec{k}_2, \vec{k}_3, \vec{k}_4) + 2\beta(\vec{k}_1, \vec{k}_2 + \vec{k}_3 + \vec{k}_4) G_3(\vec{k}_2, \vec{k}_3, \vec{k}_4)\\
& + 9\alpha(\vec{k}_1 + \vec{k}_2, \vec{k}_3 + \vec{k}_4) G_2(\vec{k}_1, \vec{k}_2)F_2(\vec{k}_3, \vec{k}_4) + 2\beta(\vec{k}_1 + \vec{k}_2, \vec{k}_3 + \vec{k}_4) G_2(\vec{k}_1, \vec{k}_2)G_2(\vec{k}_2, \vec{k}_3)\\
&+ 9\alpha(\vec{k}_1 + \vec{k}_2 + \vec{k}_3, \vec{k}_4) G_3(\vec{k}_1, \vec{k}_2, \vec{k}_3) + 2\beta(\vec{k}_1 + \vec{k}_2 + \vec{k}_3, \vec{k}_4) G_3(\vec{k}_1, \vec{k}_2, \vec{k}_3)\\
& + 23\,\text{perms.}
\end{align*}
$$

In [17]:
def F4s_reg(ks):
    result = 0
    index_list = [0,1,2,3]
    # for indices in permutations(range(4)):
    for i in range(4):
        indices = [i] + index_list[:i] + index_list[i + 1:]
        kl = np.asarray(ks)[list(indices)]
        result += (
            9*kernels.alpha(kl[0], sum(kl[1:]))*F3s_reg(kl[1:])
            + 2*kernels.beta(kl[0], sum(kl[1:]))*G3s_reg(kl[1:])
            + 9*kernels.alpha(sum(kl[1:]), kl[0])*G3s_reg(kl[1:])
            + 2*kernels.beta(sum(kl[1:]), kl[0])*G3s_reg(kl[1:])
        )/33/4
    for i in range(1,4):
        indices = [0, i] + index_list[1:i] + index_list[i + 1:]
        kl = np.asarray(ks)[list(indices)]
        if kl[0] + kl[1] != v0 and kl[2] + kl[3] != v0:
            result += (
                9*kernels.alpha(kl[0] + kl[1], kl[2] + kl[3])*kernels.Gs(2, kl[:2])*kernels.Fs(2, kl[2:])
                + 4*kernels.beta(kl[0] + kl[1], kl[2] + kl[3])*kernels.Gs(2, kl[:2])*kernels.Gs(2, kl[2:])
                + 9*kernels.alpha(kl[2] + kl[3], kl[0] + kl[1])*kernels.Gs(2, kl[2:])*kernels.Fs(2, kl[:2])
            )/33/6
    return result

Let's check that it satisfies the consistency relation

$$
\lim_{q \rightarrow 0}F_4(\bm{q}, \bm{p}, -\bm{p}, \bm{k}) = \frac{\bm{k}.\bm{q}}{4q^2} F_3(\bm{p}, -\bm{p}, \bm{k}) + \mathcal{O}(q^0)\,.
$$

In [18]:
vq, vp, vk = vect.vectors('q', 'p', 'k')
# We use c to keep track of cosines involving q
q, p, k, muqp, muqk, mukp = sp.symbols([
    "q", "p", "k", 
    r"\mu_{qp}", r"\mu_{qk}", r"\mu_{kp}"
])
f4Raw = F4s_reg([vq, vp, -vp, vk])
pdp = vect.sympy(vect.Dot(vp, vp))
kdk = vect.sympy(vect.Dot(vk, vk))
qdq = vect.sympy(vect.Dot(vq, vq))
qdp = vect.sympy(vect.Dot(vq, vp))
qdk = vect.sympy(vect.Dot(vq, vk))
pdk = vect.sympy(vect.Dot(vp, vk))
replacements = [
    (qdq, q**2), 
    (pdp, p**2), 
    (kdk, k**2), 
    (qdp, q*p*muqp),
    (qdk, q*k*muqk),
    (pdk, p*k*mukp)
]
f4 = vect.sympy(f4Raw).subs(replacements)

seriesm1 = sp.series(f4, q, n=0).removeO()
f3raw = F3s_reg([vp, -vp, vk])
f3 = vect.sympy(f3raw).subs(replacements)
rhs = sp.simplify(((muqk*k)/(4*q))*f3)
sp.simplify(seriesm1 - rhs)

0

Great! Now generate the kernel!

In [19]:
vk1, vk2, vp = vect.vectors("k1", "k2", "p")
# We use c to keep track of cosines involving q
k1, k2, mu12, p, mu1, mu2 = sp.symbols([
    "k1", "k2", "mu12", "p", "mu1", "mu2"
])
kernel_raw = F4s_reg([vk1, vk2, vp, -vp])
pdp = vect.sympy(vect.Dot(vp, vp))
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
k2dp = vect.sympy(vect.Dot(vk2, vp))
k2dk1 = vect.sympy(vect.Dot(vk2, vk1))
k1dp = vect.sympy(vect.Dot(vk1, vp))
replacements = [
    (k2dk2, k2**2), 
    (pdp, p**2), 
    (k1dk1, k1**2),  
    (k2dp, k2*p*mu2),
    (k2dk1, k1*k2*mu12),
    (k1dp, k1*p*mu1)
]
kernel_symbolic = sp.simplify(vect.sympy(kernel_raw).subs(replacements))
kernel_string = str(kernel_symbolic)
with open("b411_kernel.txt", "w") as fout:
    fout.write(kernel_string)


Now the squeezed limit kernel.

In [ ]:
vk1, vk2, vp = vect.vectors('k1', 'k2', 'p')
k1, k2, p, mu1, mu2, mu12 = sp.symbols([
    "k1", "k2", "p", 
    r"mu1", r"mu2", r"mu12"
])
f4Raw = F4s_reg([vk1, vp, -vp, vk2])
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
replacements = [
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2),
    (k1dk2, k1*k2*mu12),
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2)
]
f4 = vect.sympy(f4Raw).subs(replacements)

series = sp.series(f4, k1, n=1).removeO()
series = sp.simplify(series)
series_string = str(series)
with open("b411_kernel_squeezed.txt", "w") as fout:
    fout.write(series_string)

In [ ]:
vk1, vk2, vp = vect.vectors('k1', 'k2', 'p')
k1, k2, p, mu1, mu2, mu12 = sp.symbols([
    "k1", "k2", "p", 
    r"mu1", r"mu2", r"mu12"
])
f4Raw = F4s_reg([vk1, vp, -vp, vk2])
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
replacements = [
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2),
    (k1dk2, k1*k2*mu12),
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2)
]
f4 = vect.sympy(f4Raw).subs(replacements)

series = sp.series(f4, k1, n=1).removeO() - sp.series(f4, k1, n=0).removeO()
series = sp.simplify(series)
series_string = str(series)
with open("b411_kernel_squeezed_0.txt", "w") as fout:
    fout.write(series_string)

In [ ]:
vk1, vk2, vp = vect.vectors('k1', 'k2', 'p')
k1, k2, p, mu1, mu2, mu12 = sp.symbols([
    "k1", "k2", "p", 
    r"mu1", r"mu2", r"mu12"
])
f4Raw = F4s_reg([vk1, vp, -vp, vk2])
k1dk1 = vect.sympy(vect.Dot(vk1, vk1))
k2dk2 = vect.sympy(vect.Dot(vk2, vk2))
pdp = vect.sympy(vect.Dot(vp, vp))
k1dk2 = vect.sympy(vect.Dot(vk1, vk2))
k1dp = vect.sympy(vect.Dot(vk1, vp))
k2dp = vect.sympy(vect.Dot(vk2, vp))
replacements = [
    (k1dk1, k1**2),
    (k2dk2, k2**2),
    (pdp, p**2),
    (k1dk2, k1*k2*mu12),
    (k1dp, k1*p*mu1),
    (k2dp, k2*p*mu2)
]
f4 = vect.sympy(f4Raw).subs(replacements)

series = sp.series(f4, k1, n=1).removeO() - sp.series(f4, k1, n=0).removeO()

series_poly = sp.poly(series, mu1, mu12)

phi = sp.Symbol("phi")
phi_integral_mu1 = sp.integrate(
    sp.sqrt(1 - mu12**2)*sp.sqrt(1 - mu2**2)*sp.cos(phi) + mu2*mu12, 
(phi, 0, 2*sp.pi))
phi_integral_mu1_sq = sp.integrate(
    (sp.sqrt(1 - mu12**2)*sp.sqrt(1 - mu2**2)*sp.cos(phi) + mu2*mu12)**2, 
(phi, 0, 2*sp.pi))
phi_integral_const = 2*sp.pi

quadratic = series_poly.coeff_monomial(mu1**2)
linear = series_poly.coeff_monomial(mu1*mu12)*mu12
zeroth = series_poly.coeff_monomial(mu12**2)*mu12**2 + series_poly.coeff_monomial(1)

if (
    series_poly.coeff_monomial(mu1) != 0 
    or series_poly.coeff_monomial(mu12) != 0
    or series_poly.degree() > 2
):
    raise Exception("Polynomial has unexpected coefficients!")

result_poly = sp.poly(
    zeroth*phi_integral_const + linear*phi_integral_mu1 + quadratic*phi_integral_mu1_sq,
    mu12)

b1_kernel = result_poly.coeff_monomial(1)
bmu_kernel = result_poly.coeff_monomial(mu12**2)

if result_poly.coeff_monomial(mu12) != 0 or result_poly.degree() > 2:
    raise Exception("Unexpected coefficient in mu12 polynomial!")

b1_kernel_string = str(b1_kernel)
bmu_kernel_string = str(bmu_kernel)

with open("b411_b1_kernel.txt", "w") as fin:
    fin.write(b1_kernel_string)
with open("b411_bmu_kernel.txt", "w") as fin:
    fin.write(bmu_kernel_string)

## Generate C code

In [7]:
def load_kernel(fname):
    with open(fname, "r") as fin:
        kernel_string = fin.read()
    return sp.sympify(kernel_string)

tree_kernel = load_kernel("tree_kernel.txt")
p22_kernel = load_kernel("p22_kernel.txt")
p31_kernel = load_kernel("p31_kernel.txt")
b222_kernel = load_kernel("b222_kernel.txt")
b321I_kernel = load_kernel("b321I_kernel.txt")
b411_kernel = load_kernel("b411_kernel.txt")

tree_kernel_squeezed = load_kernel("tree_kernel_squeezed.txt")
b321I_kernel_squeezed = load_kernel("b321I_kernel_squeezed.txt")
b411_kernel_squeezed = load_kernel("b411_kernel_squeezed.txt")

tree_kernel_squeezed_0 = load_kernel("tree_kernel_squeezed_0.txt")
b321I_kernel_squeezed_0 = load_kernel("b321I_kernel_squeezed_0.txt")
b411_kernel_squeezed_0 = load_kernel("b411_kernel_squeezed_0.txt")

tree_b1_kernel = load_kernel("tree_b1_kernel.txt")
tree_bmu_kernel = load_kernel("tree_bmu_kernel.txt")
b321I_b1_kernel = load_kernel("b321I_b1_kernel.txt")
b321I_bmu_kernel = load_kernel("b321I_bmu_kernel.txt")
b411_b1_kernel = load_kernel("b411_b1_kernel.txt")
b411_bmu_kernel = load_kernel("b411_bmu_kernel.txt")

from sympy.utilities.codegen import codegen
codegen(
    [
        ('tree_kernel', tree_kernel),
        ('p22_kernel', p22_kernel),
        ('p31_kernel', p31_kernel),
        ('b222_kernel', b222_kernel),
        ('b321I_kernel', b321I_kernel),
        ('b411_kernel', b411_kernel),
        ('tree_kernel_squeezed', tree_kernel_squeezed),
        ('b321I_kernel_squeezed', b321I_kernel_squeezed),
        ('b411_kernel_squeezed', b411_kernel_squeezed),
        ('tree_kernel_squeezed_0', tree_kernel_squeezed_0),
        ('b321I_kernel_squeezed_0', b321I_kernel_squeezed_0),
        ('b411_kernel_squeezed_0', b411_kernel_squeezed_0),
        ('tree_b1_kernel', tree_b1_kernel),
        ('tree_bmu_kernel', tree_bmu_kernel),
        ('b321I_b1_kernel', b321I_b1_kernel),
        ('b321I_bmu_kernel', b321I_bmu_kernel),
        ('b411_b1_kernel', b411_b1_kernel),
        ('b411_bmu_kernel', b411_bmu_kernel)
    ],
    "C99", "kernels", to_files=True
)